# T1505.005 - Terminal Services DLL
Adversaries may abuse components of Terminal Services to enable persistent access to systems. Microsoft Terminal Services, renamed to Remote Desktop Services in some Windows Server OSs as of 2022, enable remote terminal connections to hosts. Terminal Services allows servers to transmit a full, interactive, graphical user interface to clients via RDP.(Citation: Microsoft Remote Desktop Services)

[Windows Service](https://attack.mitre.org/techniques/T1543/003)s that are run as a "generic" process (ex: <code>svchost.exe</code>) load the service's DLL file, the location of which is stored in a Registry entry named <code>ServiceDll</code>.(Citation: Microsoft System Services Fundamentals) The <code>termsrv.dll</code> file, typically stored in `%SystemRoot%\System32\`, is the default <code>ServiceDll</code> value for Terminal Services in `HKLM\System\CurrentControlSet\services\TermService\Parameters\`.

Adversaries may modify and/or replace the Terminal Services DLL to enable persistent access to victimized hosts.(Citation: James TermServ DLL) Modifications to this DLL could be done to execute arbitrary payloads (while also potentially preserving normal <code>termsrv.dll</code> functionality) as well as to simply enable abusable features of Terminal Services. For example, an adversary may enable features such as concurrent [Remote Desktop Protocol](https://attack.mitre.org/techniques/T1021/001) sessions by either patching the <code>termsrv.dll</code> file or modifying the <code>ServiceDll</code> value to point to a DLL that provides increased RDP functionality.(Citation: Windows OS Hub RDP)(Citation: RDPWrap Github) On a non-server Windows OS this increased functionality may also enable an adversary to avoid Terminal Services prompts that warn/log out users of a system when a new RDP session is created.

## Atomic Tests

### Atomic Test #1 - Simulate Patching termsrv.dllSimulates patching of termsrv.dll by making a benign change to the file and replacing it with the original afterwards.
Before we can make the modifications we need to take ownership of the file and grant ourselves the necessary permissions.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$termsrvDll = "C:\Windows\System32\termsrv.dll"

$ACL = Get-Acl $termsrvDll
$permission = "Administrators","FullControl","Allow"
$accessRule = New-Object System.Security.AccessControl.FileSystemAccessRule $permission
$ACL.SetAccessRule($accessRule)
Set-Acl -Path $termsrvDll -AclObject $ACL

Copy-Item -Path "C:\Windows\System32\termsrv.dll" -Destination "C:\Windows\System32\termsrv_backup.dll" -ErrorAction Ignore
Add-Content -Path "C:\Windows\System32\termsrv.dll" -Value "`n" -NoNewline -ErrorAction Ignore
Move-Item -Path "C:\Windows\System32\termsrv_backup.dll" -Destination "C:\Windows\System32\termsrv.dll" -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1505.005 -TestNumbers 1

#### Cleanup: 
```powershell
Move-Item -Path "C:\Windows\System32\termsrv_backup.dll" -Destination "C:\Windows\System32\termsrv.dll" -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1505.005 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Modify Terminal Services DLL PathThis atomic test simulates the modification of the ServiceDll value in HKLM\System\CurrentControlSet\services\TermService\Parameters. This technique may be leveraged by adversaries to establish persistence by loading a patched version of the DLL containing malicious code.**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$termsrvDll = "C:\Windows\System32\termsrv.dll"

$ACL = Get-Acl $termsrvDll
$permission = "Administrators","FullControl","Allow"
$accessRule = New-Object System.Security.AccessControl.FileSystemAccessRule $permission
$ACL.SetAccessRule($accessRule)
Set-Acl -Path $termsrvDll -AclObject $ACL

Copy-Item -Path $termsrvDll -Destination "$HOME\AtomicTest.dll"

$newServiceDll = "$HOME\AtomicTest.dll"

$registryPath = "HKLM:\System\CurrentControlSet\services\TermService\Parameters"

# Check if the registry key exists
if (Test-Path -Path $registryPath) {
    # Modify the ServiceDll value in the registry
    Set-ItemProperty -Path $registryPath -Name "ServiceDll" -Value $newServiceDll
    Write-Host "ServiceDll value in the registry has been updated to: $newServiceDll"
} else {
    Write-Host "Registry key not found. Make sure the 'TermService\Parameters' key exists."
}```

In [ ]:
Invoke-AtomicTest T1505.005 -TestNumbers 2

#### Cleanup: 
```powershell
Set-ItemProperty -Path "HKLM:\System\CurrentControlSet\services\TermService\Parameters" -Name "ServiceDll" -Value "C:\Windows\System32\termsrv.dll"```

In [ ]:
Invoke-AtomicTest T1505.005 -TestNumbers 2 -Cleanup

## Detection
Monitor for changes to Registry keys associated with <code>ServiceDll</code> and other subkey values under <code>HKLM\System\CurrentControlSet\services\TermService\Parameters\</code>.

Monitor unexpected changes and/or interactions with <code>termsrv.dll</code>, which is typically stored in <code>%SystemRoot%\System32\</code>.

Monitor commands as well as  processes and arguments for potential adversary actions to modify Registry values (ex: <code>reg.exe</code>) or modify/replace the legitimate <code>termsrv.dll</code>.

Monitor module loads by the Terminal Services process (ex: <code>svchost.exe -k termsvcs</code>) for unexpected DLLs (the default is <code>%SystemRoot%\System32\termsrv.dll</code>, though an adversary could also use [Match Legitimate Name or Location](https://attack.mitre.org/techniques/T1036/005) on a malicious payload).